In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
def deepnn(x):
#  """숫자를 분류하기 위한 Deep Neural Networks 그래프를 생성한다."""

  #인자들(Args):
   # x: (N_examples, 784) 차원을 가진 input tensor, 784 일반적인 MNIST 데이터의 픽셀 개수이다.
  #리턴값들(Returns):
 #   tuple (y, keep_prob). y는 (N_examples, 10)형태의 숫자(0-9) tensor이다. 
#    keep_prob는 dropout을 위한 scalar placeholder이다.

    # Convolutional Neural Netwokrs(CNNs)를 위한 reshape.
  # 마지막 차원(dimension)은 특징들("features")을 나타낸다.-이 코드에서는 이미지가 grayscale이라 일차원이지만, RGB 이미지라면 3차원, RGBA라면 4차원 이미지 일 것이다.
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # 첫번째 convolutional layer - 하나의 grayscale 이미지를 32개의 특징들(feature)으로 맵핑(maping)한다.
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - 2X만큼 downsample한다.
    h_pool1 = max_pool_2x2(h_conv1)

  # 두번째 convolutional layer -- 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)한다.
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # 두번째 pooling layer.
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully Connected Layer 1 -- 2번의 downsampling 이후에, 우리의 28x28 이미지는 7x7x64 특징들(feature map)이 된다.
  # 이를 1024개의 특징들로 맵핑(maping)한다.
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
mnist=input_data.read_data_sets('./mnist', one_hot=True)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [5]:
X=tf.placeholder(tf.float32, [None, 784])
y_=tf.placeholder(tf.float32, [None, 10])

In [6]:
y_conv, keep_prob=deepnn(X)

In [7]:
cross_entropy=tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [8]:
correct_prediction=tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        batch=mnist.train.next_batch(50)
        if i%10==0:
            train_accuracy=accuracy.eval(feed_dict={
                X:batch[0], y_:batch[1], keep_prob:1.0
            })
            print('step %d, training accuracy %g' %(i, train_accuracy))
        train_step.run(feed_dict={X:batch[0], y_:batch[1], keep_prob:0.5})
    
    print('test_accuracy %g' %(accuracy.eval(feed_dict={
        X:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})))

step 0, training accuracy 0.06
step 10, training accuracy 0.34
step 20, training accuracy 0.52
step 30, training accuracy 0.62
step 40, training accuracy 0.68
step 50, training accuracy 0.68
step 60, training accuracy 0.82
step 70, training accuracy 0.72
step 80, training accuracy 0.84
step 90, training accuracy 0.9
step 100, training accuracy 0.92
step 110, training accuracy 0.84
step 120, training accuracy 0.84
step 130, training accuracy 0.84
step 140, training accuracy 0.84
step 150, training accuracy 0.82
step 160, training accuracy 0.9
step 170, training accuracy 0.8
step 180, training accuracy 0.9
step 190, training accuracy 0.9
step 200, training accuracy 0.88
step 210, training accuracy 0.88
step 220, training accuracy 0.92
step 230, training accuracy 0.88
step 240, training accuracy 0.98
step 250, training accuracy 0.96
step 260, training accuracy 0.86
step 270, training accuracy 0.9
step 280, training accuracy 0.94
step 290, training accuracy 0.9
step 300, training accuracy 